In [1]:
from mycelia.shared.config import MinerConfig, ValidatorConfig
from mycelia.shared.cycle import *
from mycelia.shared.chain import * 
from mycelia.miner.train import * 
from mycelia.miner.model_io import *
import bittensor as bt 


PyTorch version 2.9.1 available.


/home/isabella/crucible/subnet-MoE/.venv/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
subtensor = bt.Subtensor('test')

path = "/home/isabella/crucible/subnet-MoE/checkpoints/miner/miner/hk1/foundation/config.yaml"
config = MinerConfig.from_path(path)

In [3]:
rank = 2
wallet, subtensor = setup_chain_worker(config)
device = torch.device(f"cuda:{rank}" if torch.cuda.is_available() else "cpu")
tokenizer = get_base_tokenizer(config)
expert_manager = ExpertManager(config)

2025-12-14 21:35:22 [info     ] load_expert_group_assignment - folder [mycelia.shared.expert_manager] positional_args=(PosixPath('/home/isabella/crucible/subnet-MoE/expert_groups/exp_dummy'),)
2025-12-14 21:35:22 [info     ] load_expert_group_assignment - folder [mycelia.shared.expert_manager] positional_args=(PosixPath('/home/isabella/crucible/subnet-MoE/expert_groups/exp_math'),)
2025-12-14 21:35:22 [info     ] load_expert_group_assignment - folder [mycelia.shared.expert_manager] positional_args=(PosixPath('/home/isabella/crucible/subnet-MoE/expert_groups/exp_math'),)


In [4]:
from mycelia.shared.cycle import *

def get_validator_seed_from_commit(config, commits):

    for c in commits:
        if isinstance(c[0], ValidatorChainCommit):
            print("Expert Group:", getattr(c[0], "expert_group", None), "Config Expert Group ID:", config.task.expert_group_id)
    validator_seeds: dict[str, int] = {
        neuron.hotkey: commit.miner_seed
        for commit, neuron in commits
        if isinstance(commit, ValidatorChainCommit)
        and getattr(commit, "expert_group", None) == config.task.expert_group_id
    }
    return validator_seeds

def get_validator_miner_assignment(config: WorkerConfig, subtensor: bittensor.Subtensor):
    commits: tuple[WorkerChainCommit, bittensor.Neuron] = get_chain_commits(config, subtensor)

    for c in commits:
        print("Commit:", c)

    validator_seeds = get_validator_seed_from_commit(config, commits)

    for v in validator_seeds:
        print("Validator Seed:", v)

    miners = get_miners_from_commit(config, commits)

    for m in miners:
        print("Miner:", m)

    return assign_miners_to_validators(validator_seeds, miners)  # type: i


def search_model_submission_destination(
    wallet: bittensor.Wallet, config: MinerConfig, subtensor: bittensor.Subtensor
) -> bittensor.Axon:
    validator_miner_assignment = get_validator_miner_assignment(config, subtensor)

    print("Validator to Miner Assignment:", validator_miner_assignment)
    for validator, miners in validator_miner_assignment.items():
        if wallet.hotkey.ss58_address in miners:
            assigned_validator_hotkey = validator
            break


    metagraph = subtensor.metagraph(netuid=config.chain.netuid)
    uid = metagraph.hotkeys.index(assigned_validator_hotkey)
    return metagraph.axons[uid]

search_model_submission_destination(
    wallet,
    config,
    subtensor,
)

decryopting status dict: {'model_hash': 'xxx', 'model_version': -1, 'expert_group': 0, 'miner_seed': 5718967, 'encrypted': -1}
2025-12-14 21:10:28 [warning  ] Failed to decrypt data       [mycelia.shared.chain] error=fromhex() argument must be str, not int
decryopting status dict: {'model_hash': 'xxx', 'model_version': -1, 'expert_group': 0, 'miner_seed': 5718967, 'encrypted': -1}
2025-12-14 21:10:28 [warning  ] Failed to decrypt data       [mycelia.shared.chain] error=non-hexadecimal number found in fromhex() arg at position 0
decryopting status dict: {'model_hash': 'xxx', 'model_version': 94, 'expert_group': 1, 'miner_seed': 3024650}
2025-12-14 21:10:28 [warning  ] Failed to decrypt data       [mycelia.shared.chain] error=fromhex() argument must be str, not int
decryopting status dict: {'model_hash': 'xxx', 'model_version': 94, 'expert_group': 1, 'miner_seed': 3024650}
2025-12-14 21:10:28 [warning  ] Failed to decrypt data       [mycelia.shared.chain] error=non-hexadecimal number fou

AxonInfo( /ipv4/149.137.225.62:8000, 5DoHdXfDYraqPzkLjrXGMZxvGXYdDYhuC8tGbQdb4zvz2LbH, 5EXGTzS8r36MrKuYGknxmyqsX4eModUaSWJKn1eXVqeoBJRr, 10000000 )

In [ ]:

_, _, latest_checkpoint_path = get_resume_info(rank=0, config=config)

model_path = f"{latest_checkpoint_path}/model.pt"
model_hash = get_model_hash(compile_full_state_dict_from_path(model_path)).hex()

2025-12-14 21:37:58 [info     ] Looking for checkpoint from folder [mycelia.shared.checkpoint] path={PosixPath('/home/isabella/crucible/subnet-MoE/checkpoints/miner/miner/hk1/foundation')} result=found
2025-12-14 21:38:07 [info     ] loaded checkpoint file       [mycelia.shared.checkpoint] loss=11.88102 path=<OpenFile '/home/isabella/crucible/subnet-MoE/checkpoints/miner/miner/hk1/foundation/globalver_-1_inneropt_40/model.pt'>
2025-12-14 21:38:07 [info     ] loaded checkpoint file       [mycelia.shared.checkpoint] loss=11.88102 path=<OpenFile '/home/isabella/crucible/subnet-MoE/checkpoints/miner/miner/hk1/foundation/globalver_-1_inneropt_40/model.pt'>
A model hash to commit: 2fda49f38a7d235eb0c41b3e0149af1a6992d09be2af8218ee39ef209a47badf
2025-12-14 21:38:40 [info     ] B Preparing to commit status to chain model hash [mycelia.shared.chain] data_dict=None
2025-12-14 21:38:40 [info     ] C after encryption           [mycelia.shared.chain] data_dict=None
A model hash to commit: 2fda49f38

In [13]:
sign_message(origin_hotkey=wallet.hotkey, message=model_hash)

'3a38a9aa92bf6ffcb56092c662e00a2a82df6ab04eb3d4e601a347cac93e847d5f8ea82ff4dcaa9be9d0b9b6d5092f29cb236d276f636446f039b20955030788'

In [ ]:

print('A model hash to commit:', model_hash)
current_block = subtensor.block

n_blocks = 1

print('current_block', current_block)

commited_message = commit_status(
    config,
    wallet,
    subtensor,
    MinerChainCommit(
        expert_group=config.task.expert_group_id,
        model_hash=model_hash,
        signed_model_hash=sign_message(origin_hotkey=wallet.hotkey, message=model_hash)[:10],
    ),
    encrypted=True,
    n_blocks=n_blocks,
)

logger.info(f"Committed with hash: {commited_message}.")

A model hash to commit: 2fda49f38a7d235eb0c41b3e0149af1a6992d09be2af8218ee39ef209a47badf
current_block 6038144
2025-12-14 22:00:57 [info     ] B Preparing to commit status to chain model hash [mycelia.shared.chain] data_dict=None
2025-12-14 22:00:57 [info     ] C after encryption           [mycelia.shared.chain] data_dict=None
2025-12-14 22:00:57 [info     ] C after encryption           [mycelia.shared.chain] data_dict=None
2025-12-14 22:00:57 [info     ] Committing status to chain   [mycelia.shared.chain] data_dict={'expert_group': 0, 'model_hash': None, 'signed_model_hash': '5c88404f3b', 'encrypted': 6038145}
2025-12-14 22:00:57 [info     ] Committing status to chain   [mycelia.shared.chain] data_dict={'expert_group': 0, 'model_hash': None, 'signed_model_hash': '5c88404f3b', 'encrypted': 6038145}
2025-12-14 22:01:26 [info     ] Committed with hash: {'expert_group': 0, 'model_hash': None, 'signed_model_hash': '5c88404f3b', 'encrypted': 6038145}. [mycelia.miner.model_io]
2025-12-14 22:

In [26]:
all_commitments = subtensor.get_all_commitments(netuid=config.chain.netuid, block = current_block)

all_commitments

{'5DoHdXfDYraqPzkLjrXGMZxvGXYdDYhuC8tGbQdb4zvz2LbH': '{"model_hash": "xxx", "model_version": -1, "expert_group": 0, "miner_seed": 5718967, "encrypted": -1}',
 '5DUqbhkpvth1WzRogVU9NvGeV9bwX3wBHFWHMZk72WEhZjHF': '{"model_hash":"xxx","model_version":94,"expert_group":1,"miner_seed":3024650}',
 '5CtaFaSaJjKgbh1sqgR43knv2rq8RfzWHDPWHxNdiM1b1NRN': '"{\\"c\\":\\"bafkreidfmlxywiqs27jf3rwjphsmhy6wrfxta5tjtxivftxxdwalkybmga\\",\\"e\\":797981,\\"pe\\":797982,\\"v\\":4}"',
 '5FnRrHZ2KDecJzytUdwz4sfBd4YW6JReYNZTHK7xRBM7jqx9': '{"expert_group": 0, "model_hash": null, "signed_model_hash": "1eecbe2854", "encrypted": 6038130}',
 '5DqZE7ktQVt1fn6cxbv6yzutxAvbQkGbmxArwKPKtoBYc8JB': '{"expert_group":1}'}